##Задание

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

Скачиваем данные

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
!ls

gazeta_test.txt    gazeta_train.txt    gazeta_val.txt	 sample_data
gazeta_test.txt.1  gazeta_train.txt.1  gazeta_val.txt.1
gazeta_test.txt.2  gazeta_train.txt.2  gazeta_val.txt.2


In [3]:
!pip install razdel networkx pymorphy2[fast] nltk rouge==0.3.1 summa lexrank sumy
!pip install --upgrade datasets razdel spacy networkx seaborn scipy tqdm tensorflow-text
!python -m spacy download ru_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-08-03 02:41:53.778488: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_md-3.4.0/ru_core_news_md-3.4.0-py3-none-any.whl (41.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


Функция, которая загружает и сохраняет данные из скачанных файлов

In [4]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

Сформируем списки с данными из файлов

In [5]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

Загрузим transformers

In [9]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


В разделе summarization на Huggingface последняя русскоязычная модель для саммаризации на русском - "csebuetnlp/mT5_multilingual_XLSum"

Попробуем использовать ее.

In [10]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down, the company said.  The policy includes the termination of accounts of anti-vaccine influencers.  Tech giants have been criticised for not doing more to counter false health information on their sites.  In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


YouTube has banned thousands of videos spreading misinformation about Covid vaccines.


Создадим функцию, которая будет использовать данную модель для возврата саммари по загруженному в нее тексту

In [57]:
def summarizator(tokenizer, model, article_text):
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
)

    return summary

###Сначала посмотрим несколько примеров глазами

Возьмем первую статью из тестового набора

In [56]:
article_text = test_records[0]['text']
article_text

'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, как они и им подобные объекты образовались в космосе». Каждый проект из выбранн

Такое саммари предлагается как референс

In [60]:
test_records[0]['summary']

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

Такое саммари выдает модель

In [59]:
summarizator(tokenizer, model, article_text)

'Космические миссии, которые могут дать ответы на вопросы о происхождении тел Солнечной системы, будут запущены в 2021 году.'

Посмотрим таким же образом еще несколько примеров

In [61]:
article_text = test_records[1]['text']
article_text

'Около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре Бурятии. В Кремле выступил Бурятский государственный академический театр оперы и балета, Национальный цирк, Бурятский национальный театр песни и танца «Байкал», ставший победителем шоу «Танцуют все!» на телеканале «Россия», а также другие профессиональные и самодеятельные коллективы региона. Более 300 артистов из одного региона на главной сцене страны - похоже это рекорд России. Зрителям рассказали, что Республике Бурятия, чье население составляет 1 миллион человек, сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков. И все они были представлены в Москве. Как писали после шоу зрители в соцсетях: «А мы думали, что в Бурятии только буряты живут…». Для неподготовленного зрителя это вообще были вечера открытий. Например, когда еще в Кремлевском дворце выстраивалась очередь из желающих попасть на прием к врачам-пульсологам и ламам-астр

In [62]:
test_records[1]['summary']

'25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.'

In [63]:
summarizator(tokenizer, model, article_text)

'В воскресенье в Москве состоялось праздничное шоу, на котором можно было увидеть все самое лучшее, что есть на сегодняшний день в культуре Бурятии.'

In [64]:
article_text = test_records[28]['text']
article_text

'Сегодня, 20 января, Совет Государственной думы рассмотрит вопрос о включении в повестку на 21 января создание должности зампредседателя Совета безопасности. Ранее на этот пост был предложен бывший российский премьер-министр Дмитрий Медведев, правительство которого ушло в отставку 15 января. Законопроект о создании новой должности был направлен в комитет по безопасности и противодействию коррупции в конце прошлой недели. Как отмечается на официальном сайте Госдумы, заместитель председателя Совбеза назначается лично президентом России. Инициативу о введении новой должности внес лично президент России Владимир Путин. О намерении сделать это он заявил еще 15 января, подчеркнув, что Медведев станет его непосредственным заместителем. Позже Путин также включил в состав Совета безопасности нового премьер-министра Михаила Мишустина. «Считаю возможным и сделаю это в ближайшее время — введу должность заместителя председателя Совета безопасности. Председателем Совета безопасности, как известно, я

In [65]:
test_records[28]['summary']

'20 января состоится заседание Совета Госдумы, который рассмотрит вопрос о включении в повестку нижней палаты парламента на 21 января создание должности зампредседателя Совета безопасности РФ, которую займет бывший премьер-министр Дмитрий Медведев. Некоторые эксперты трактуют это как понижение, однако на деле Медведев фактически станет правой рукой Владимира Путина в сфере безопасности и вторым лицом одного из важнейших совещательных органов России.'

In [66]:
summarizator(tokenizer, model, article_text)

'Президент России Владимир Путин внес в Госдуму законопроект о введении нового зампредседателя Совета безопасности.'

In [67]:
article_text = test_records[186]['text']
article_text

'Переговоры между Багдадом и Москвой о покупке Ираком российских зенитных комплексов С-300 возобновились, сообщил глава комитета по обороне и безопасности парламента Ирака Мухаммад Рида в комментарии РИА «Новости», отметив, что этот вопрос должен был решиться давно. «Несколько месяцев назад после атак на базы шиитского ополчения «Аль-Хашд аш-Шааби» в Багдаде и других провинциях возникла необходимость в этом оружии, и ответственная сторона возобновила переговоры по контракту на С-300», — рассказал собеседник агентства. Рида добавил, что не знает, на каком этапе находится сделка в настоящее время, но он может сказать, что ее санкционировало руководство Ирака. «Мы ожидаем противодействия со стороны США в этом вопросе», — обратил внимание парламентарий. По его словам, ранее переговоры были приостановлены из-за того, что Вашингтон угрожал санкциями всем покупателям С-300. Вместе с тем чиновник отметил, что у Ирака есть уже некоторые российские системы. Тем не менее власти страны хотят приоб

In [68]:
test_records[186]['summary']

'В Ираке заявили о «возникшей необходимости» в покупке у России зенитных комплексов С-300. Несмотря на то, что у страны уже есть российское вооружение для ПВО, власти хотят приобрести более современное оружие. При этом США ранее обещали ввести санкции против покупателей российского оружия.'

In [69]:
summarizator(tokenizer, model, article_text)

'Переговоры между Багдадом и Москвой о покупке Ираком российских зенитных комплексов С-300 возобновились, сообщил министр обороны Ирака.'

Если смотреть глазами, то в целом саммаризация достаточно осмысленная, но очень короткая. Думаю, что метрики будут низкими.

##Попробуем теперь оценить метрики BLEU и ROUGE для данной модели

Создадим функции, которые позволят оценить метрики по корпусу статей

In [48]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])


    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

Т.к. модель отрабатывает очень долго, оценим метрики по корпусу из 10 статей

In [54]:
import razdel

def calc_model_score(records, model, tokenizer, model_name, lower=True, nrows=10):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

        article_text = record["text"]
        input_ids = tokenizer(
            [WHITESPACE_HANDLER(article_text)],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"]

        output_ids = model.generate(
            input_ids=input_ids,
            max_length=84,
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]

        summary = tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        predictions.append(summary)

    return calc_scores(references, predictions)

calc_model_score(test_records, model, tokenizer, model_name)

Count: 10
Ref: работающие пенсионеры дождались индексации. это следует из уже одобренных в первом чтении поправок в конституцию. так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. в россии их насчитывается более девяти миллионов. эксперты называют это восстановлением справедливости.
Hyp: Госдума одобрила в первом чтении внесенные президентом России Владимиром Путиным поправки в Конституцию. В них легли предложения об индексации пенсий работающим пенсионерам, в том числе с инвалидностью.
BLEU:  0.15452866886544483
ROUGE:  {'rouge-1': {'f': 0.16642777547269708, 'p': 0.2673042398777693, 'r': 0.12304875638641963}, 'rouge-2': {'f': 0.04901111250924249, 'p': 0.08456022877378375, 'r': 0.035078751952987916}, 'rouge-l': {'f': 0.12342721029971511, 'p': 0.2437339784398608, 'r': 0.1123414793629105}}


Как и предполагалось, метрики достаточно низкие. При этом по факту саммаризация достаточно осмысленная, но короткая относительно референсов.

К сожалению, времени на второе задание нет. Оно указано как дополнительное.